#Восстановление пропущенных значений 
По разным причинам многие наборы данных реального мира содержат пропущенные значения, часто закодированные как пробелы, NaN или другие заполнители. Однако такие наборы данных несовместимы с оценщиками scikit-learn, которые предполагают, что все значения в массиве являются числовыми, и что все они имеют и имеют значение. Основная стратегия использования неполных наборов данных — отбрасывать целые строки и / или столбцы, содержащие пропущенные значения. Однако это происходит ценой потери данных, которые могут быть ценными (хотя и неполными). Лучшая стратегия — это вменять недостающие значения, т.е. вывести их из известной части данных.

#Импутация данных средним/медианой
Первый спосб имутации данных - это подсчитать среднее или медиану имеющихся значений для каждого столбца в таблице и вставляем то, что получилось, в пустые ячейки. Естественно, этот метод работает только с численными данными.

##Плюсы:
Просто и быстро.

Хорошо работает на небольших наборах численных данных.
###Минусы:
Значения вычисляются независимо для каждого столбца, так что корреляции между параметрами не учитываются.

Не работает с качественными переменными.

Метод не особенно точный.

Никак не оценивается погрешность импутации.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from math import sqrt
import random
import numpy as np
random.seed(0)

# Скачиваем данные:
import pandas as pd
dataset = fetch_california_housing()
train, target = pd.DataFrame(dataset.data), pd.DataFrame(dataset.target)
train.columns = ['0','1','2','3','4','5','6','7']
train.insert(loc=len(train.columns), column='target', value=target)

# Случайным образом заменяем 40% значений на NaN
column = train['0']
print(column.size)
missing_pct = int(column.size * 0.4)
i = [random.choice(range(column.shape[0])) for _ in range(missing_pct)]
column[i] = np.NaN
print(column.shape[0])

# восстанавливаем удалённое с помощью класса SimpleImputer из scikit-learn
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer( strategy='mean') # для импутации медианой замените 'mean' на 'median'
imp_mean.fit(train)
imputed_train_df = imp_mean.transform(train)
imputed_train_df

20640
20640


array([[   3.87793988,   41.        ,    6.98412698, ...,   37.88      ,
        -122.23      ,    4.526     ],
       [   8.3014    ,   21.        ,    6.23813708, ...,   37.86      ,
        -122.22      ,    3.585     ],
       [   3.87793988,   52.        ,    8.28813559, ...,   37.85      ,
        -122.24      ,    3.521     ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,   39.43      ,
        -121.22      ,    0.923     ],
       [   3.87793988,   18.        ,    5.32951289, ...,   39.43      ,
        -121.32      ,    0.847     ],
       [   2.3886    ,   16.        ,    5.25471698, ...,   39.37      ,
        -121.24      ,    0.894     ]])

#Импутация данных самым частым значением или константой
  Импутация самым часто встречающимся значением — ещё одна простая стратегия для компенсации пропущенных значений, не учитывающая корреляций между параметрами. Плюсы и минусы те же, что и в предыдущем пункте, но этот метод предназначен для качественных переменных.
  В случае импутации нулём или константой (как можно догадаться из названия) все пропущенные значения в данных заменяются определённым значением.

In [2]:
# Импутация классом SimpleImputer из scikit-learn

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer( strategy='most_frequent')
imp_mean.fit(train)
imputed_train_df = imp_mean.transform(train)
imputed_train_df

array([[   3.125     ,   41.        ,    6.98412698, ...,   37.88      ,
        -122.23      ,    4.526     ],
       [   8.3014    ,   21.        ,    6.23813708, ...,   37.86      ,
        -122.22      ,    3.585     ],
       [   3.125     ,   52.        ,    8.28813559, ...,   37.85      ,
        -122.24      ,    3.521     ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,   39.43      ,
        -121.22      ,    0.923     ],
       [   3.125     ,   18.        ,    5.32951289, ...,   39.43      ,
        -121.32      ,    0.847     ],
       [   2.3886    ,   16.        ,    5.25471698, ...,   39.37      ,
        -121.24      ,    0.894     ]])

#Импутация данных с помощью k-NN
k-Nearest Neighbour (k ближайших соседей) — простой алгоритм классификации, который можно модифицировать для импутации недостающих значений. Он использует сходство точек, чтобы предсказать недостающие значения на основании k ближайших точек, у которых это значение есть. Иными словами, выбирается k точек, которые больше всего похожи на рассматриваемую, и уже на их основании выбирается значение для пустой ячейки. Это можно сделать с помощью библиотеки Impyute.
Алгоритм сперва проводит импутацию простым средним, потом на основании получившегося набора данных строит дерево и использует его для поиска ближайших соседей. Взвешенное среднее их значений и вставляется в исходный набор данных вместо недостающего.
##Плюсы:
На некоторых датасетах может быть точнее среднего/медианы или константы.

Учитывает корреляцию между параметрами.
###Минусы:
Вычислительно дороже, так как требует держать весь набор данных в памяти.

Важно понимать, какая метрика дистанции используется для поиска соседей.

Имплементация в impyute поддерживает только манхэттенскую и евклидову дистанцию, так что анализ соотношений (скажем, количества входов на сайты людей разных возрастов) может потребовать предварительной нормализации.

Чувствителен к выбросам в данных (в отличие от SVM).

In [3]:
!pip install impyute
import sys
from impyute.imputation.cs import fast_knn

# Увеличиваем максимальную глубину рекурсии
sys.setrecursionlimit(100000)

# начинаем тренировку KNN
imputed_training=fast_knn(train.values, k=30)

imputed_training

array([[   3.30117882,   41.        ,    6.98412698, ...,   37.88      ,
        -122.23      ,    4.526     ],
       [   8.3014    ,   21.        ,    6.23813708, ...,   37.86      ,
        -122.22      ,    3.585     ],
       [   4.31326938,   52.        ,    8.28813559, ...,   37.85      ,
        -122.24      ,    3.521     ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,   39.43      ,
        -121.22      ,    0.923     ],
       [   3.40189346,   18.        ,    5.32951289, ...,   39.43      ,
        -121.32      ,    0.847     ],
       [   2.3886    ,   16.        ,    5.25471698, ...,   39.37      ,
        -121.24      ,    0.894     ]])

#Импутация данных с помощью глубокого обучения
Показано, что глубокое обучение хорошо работает с дискретными и другими не-численными значениями. Библиотека datawig позволяет восстанавливать недостающие значения за счёт тренировки нейросети на тех точках, для которых есть все параметры. Поддерживается тренировка на CPU и GPU.

##Плюсы:
Точнее других методов.
Может работать с качественными параметрами.
Поддерживает CPU и GPU.
###Минусы:
Восстанавливает только один столбец.

На больших наборах данных может быть вычислительно дорого.

Нужно заранее решить, какие столбцы будут использоваться для предсказания недостающего значения.

In [4]:
!pip install datawig
import datawig

df_train, df_test = datawig.utils.random_split(train)

# Инициализируем модель SimpleImputer
imputer = datawig.SimpleImputer(
	input_columns=['1','2','3','4','5','6','7', 'target'], # из каких столбцов брать информацию для импутации
	output_column= '0', # какой столбец восстанавливаем
	output_path = 'imputer_model' # куда записывать модель и её метрики
	)

# Тренируем модель
imputer.fit(train_df=df_train, num_epochs=50)

# Проводим импутацию и возвращаем восстановленный набор данных
imputed = imputer.predict(df_test)

2021-08-04 20:16:25,614 [INFO]  
========== start: fit model
2021-08-04 20:16:25,616 [WARNING]  Already bound, ignoring bind()
2021-08-04 20:16:26,224 [INFO]  Epoch[0] Batch [0-465]	Speed: 12461.31 samples/sec	cross-entropy=9.262596	0-accuracy=0.000000
2021-08-04 20:16:26,807 [INFO]  Epoch[0] Train-cross-entropy=8.884165
2021-08-04 20:16:26,810 [INFO]  Epoch[0] Train-0-accuracy=0.000000
2021-08-04 20:16:26,812 [INFO]  Epoch[0] Time cost=1.191
2021-08-04 20:16:26,824 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2021-08-04 20:16:26,865 [INFO]  Epoch[0] Validation-cross-entropy=7.476050
2021-08-04 20:16:26,867 [INFO]  Epoch[0] Validation-0-accuracy=0.000000
2021-08-04 20:16:27,458 [INFO]  Epoch[1] Batch [0-465]	Speed: 12766.86 samples/sec	cross-entropy=13.684176	0-accuracy=0.000000
2021-08-04 20:16:28,037 [INFO]  Epoch[1] Train-cross-entropy=12.796621
2021-08-04 20:16:28,038 [INFO]  Epoch[1] Train-0-accuracy=0.000000
2021-08-04 20:16:28,044 [INFO]  Epoch[1] Time cost=1.17

#Одномерное восстановление признаков
SimpleImputer предоставляет основные стратегии для восстановления отсутствующих значений. Пропущенные значения могут быть восстановлены с использованием предоставленного постоянного значения или с использованием статистики (среднего, медианного или наиболее частого) каждого столбца, в котором находятся пропущенные значения. Этот класс также допускает различные кодировки пропущенных значений.

Следующий фрагмент демонстрирует, как заменить отсутствующие значения, закодированные как np.nan, с использованием среднего значения столбцов (ось 0), содержащих отсутствующие значения:

In [5]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit([[1, 2], [np.nan, 3], [7, 6]])
SimpleImputer()
X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))

[[4.         2.        ]
 [6.         3.66666667]
 [7.         6.        ]]


SimpleImputer также поддерживает разреженные матрицы (матрица с преимущественно нулевыми элементами. В противном случае, если большая часть элементов матрицы ненулевые, матрица считается плотной):

In [6]:
import scipy.sparse as sp
X = sp.csc_matrix([[1, 2], [0, -1], [8, 4]])
imp = SimpleImputer(missing_values=-1, strategy='mean')
imp.fit(X)
SimpleImputer(missing_values=-1)
X_test = sp.csc_matrix([[-1, 2], [6, -1], [7, 6]])
print(imp.transform(X_test).toarray())

[[3. 2.]
 [6. 3.]
 [7. 6.]]


Стоит обратить внимание, что этот формат не предназначен для использования для неявного хранения отсутствующих значений в матрице, поскольку он уплотняет ее во время преобразования. Отсутствующие значения, закодированные с помощью 0, должны использоваться с плотным вводом.

Класс SimpleImputer также поддерживает категорические данные , представленные в виде строковых значений или панд categoricals при использовании 'most_frequent' или 'constant' стратегии:

In [7]:
import pandas as pd
df = pd.DataFrame([["a", "x"],
                   [np.nan, "y"],
                   ["a", np.nan],
                   ["b", "y"]],
                  dtype="category")

imp = SimpleImputer(strategy="most_frequent")
print(imp.fit_transform(df))

[['a' 'x']
 ['a' 'y']
 ['a' 'y']
 ['b' 'y']]
